Type project: Individual

**Deadline**:11.12.2025 23:59:00

Max possible score: 100

In [2]:
import pandas as pd

In [4]:
data = pd.read_csv('t_zz_text.csv', on_bad_lines='warn')

C:\Users\Ser1kbay_Aza\AppData\Local\Temp\ipykernel_21668\721890436.py:1: ParserWarning: Skipping line 282: expected 3 fields, saw 5
Skipping line 809: expected 3 fields, saw 5
Skipping line 1338: expected 3 fields, saw 5
Skipping line 1339: expected 3 fields, saw 5
Skipping line 1340: expected 3 fields, saw 5
Skipping line 1745: expected 3 fields, saw 5
Skipping line 1849: expected 3 fields, saw 5
Skipping line 1967: expected 3 fields, saw 5
Skipping line 2158: expected 3 fields, saw 5
Skipping line 2177: expected 3 fields, saw 5
Skipping line 2178: expected 3 fields, saw 5
Skipping line 2458: expected 3 fields, saw 5
Skipping line 3044: expected 3 fields, saw 5
Skipping line 3275: expected 3 fields, saw 5
Skipping line 3388: expected 3 fields, saw 5
Skipping line 3671: expected 3 fields, saw 5
Skipping line 4019: expected 3 fields, saw 5
Skipping line 4374: expected 3 fields, saw 5
Skipping line 4656: expected 3 fields, saw 5
Skipping line 4801: expected 3 fields, saw 5
Skipping line 

In [5]:
data.head()

,,|idchain|datetimestart|datetimestop|lentime|month_|name_1|name_2|name_3|comb_name|call_status|overall_silence|talk_language|transcript_operator_words
0|01MMQUNSFK9KDFVKR0F0U2LAES05AASG|2022-12-22 05:01:04|2022-12-22 05:05:13|243.0|12.2022|КРЕДИТЫ|ДЕНЕЖНЫЕ КРЕДИТЫ|ПРОДАЖА \ УСЛОВИЯ ПО НОВОМУ КД|КРЕДИТЫ,ДЕНЕЖНЫЕ КРЕДИТЫ,ПРОДАЖА \ УСЛОВИЯ ПО НОВОМУ КД||43.85|kz|здра...
1|01MMQUNSFK9KDFVKR0F0U2LAES05CDPU|2022-12-22 13:59:01|2022-12-22 14:04:34|332.0|12.2022|КАРТА|КРЕДИТНАЯ КАРТА|СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД|КАРТА,КРЕДИТНАЯ КАРТА,СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД||1.8|kz...
2|01MMQUNSFK9KDFVKR0F0U2LAES05CDPU|2022-12-22 13:59:01|2022-12-22 14:04:34|332.0|12.2022|КАРТА|КРЕДИТНАЯ КАРТА|СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД|КАРТА,КРЕДИТНАЯ КАРТА,СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД||5.5000...
3|01MMQUNSFK9KDFVKR0F0U2LAES05CDPU|2022-12-22 13:59:01|2022-12-22 14:04:34|332.0|12.2022|КАРТА|КРЕДИТНАЯ КАРТА|СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД|КАРТА,КРЕДИТНАЯ КАРТА,СОСТОЯНИЕ\ УСЛОВИЯ ПО ДЕЙСТВУЮЩЕМУ КД||51.610...
4|01MMQUNSFK9KDFVKR0F0U2LAES05BEOQ|2022-12-22 09:29:26|2022-12-22 09:30:09|38.0|12.2022|ТРЕТЬЕ ЛИЦО|ИДЕНТИФИКАЦИЯ\ОТКАЗ ОТ ИНФО|ОТКАЗ ОТ ИНФО|ТРЕТЬЕ ЛИЦО,ИДЕНТИФИКАЦИЯ\ОТКАЗ ОТ ИНФО,ОТКАЗ ОТ ИНФО||0.0|ru|сәлеметсіз бе здравству...


In [ ]:
#  No specific task, only dataset.
#  Define the task by your own: Classification, Clusterization, Embedding, Languages Models..

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import re
import warnings
warnings.filterwarnings('ignore')

rows = []
with open('t_zz_text.csv', 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('|')
        if len(parts) >= 14:
            rows.append(parts[1:14])

data = pd.DataFrame(rows[1:], columns=['idchain', 'datetime_start', 'datetime_stop', 'lentime', 
                                        'month', 'name_1', 'name_2', 'name_3', 'comb_name', 
                                        'call_status', 'overall_silence', 'talk_language', 'transcript'])

data = data[['name_1', 'transcript']].dropna()
print(f"{data.shape[0]} строк")

402092 строк


In [38]:

data['text'] = data['transcript'].str.lower()
data = data[data['text'].str.len() > 30]

# топ-4 класса для баланса точности и разделимости
top4 = data['name_1'].value_counts().head(4).index
df = data[data['name_1'].isin(top4)][['name_1', 'text']].copy()

print(f"{df.shape[0]} строк")
print(df['name_1'].value_counts())

317174 строк
name_1
КРЕДИТЫ         145165
ПЕРЕКЛЮЧЕНИЯ     89791
КАРТА            51023
ТРЕТЬЕ ЛИЦО      31195
Name: count, dtype: int64


In [40]:
#
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['name_1'], test_size=0.15, random_state=42, stratify=df['name_1']
)

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,3), min_df=3, max_df=0.8)
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)
print(f"Train: {X_train_vec.shape}")

Train: (269597, 5000)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(C=10, max_iter=1000, solver='lbfgs', random_state=42)
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")

Accuracy: 0.678
